# 04_summary

In [ ]:
# 1) total number of patients in the cohort

In [ ]:
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import timedelta
import traceback

current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
with open(parent_dir.joinpath("config.json")) as file:
    cfg = json.load(file)
with open(current_dir.joinpath("demographics.json")) as file:
    dg_cfg = json.load(file)

In [ ]:
import pathlib
result_dir = current_dir.joinpath("result")
with open(result_dir.joinpath("number_of_patients.json")) as file:
    summary = json.load(file)

In [ ]:
# n_patients_df = pd.DataFrame()
# eject n total patients 
if "n_total_patients" in summary:
    n_total_patients = summary.pop("n_total_patients")

n_patients_df = pd.DataFrame.from_dict(summary, orient="index")
n_patients_df.rename(columns={"False": "control", "True": "case"}, inplace=True)
n_patients_df

In [ ]:
with open(result_dir.joinpath("demographics_for_condition_occurrence.json")) as file:
    demographics_condition = json.load(file)
df_dict = {}
for condition in demographics_condition:
    cond_df = pd.DataFrame.from_dict(demographics_condition[condition], orient="index")
    cond_df.rename(columns={"False": "control", "True": "case"}, inplace=True)
        # cond_df['case_rate'] = cond_df['case'] / n_patients_df.loc[condition, 'total']
        # cond_df['control_rate'] = cond_df['control'] / n_patients_df.loc[condition, 'total']
    df_dict[condition] = cond_df
df_dict

In [ ]:
concat_df = pd.concat(df_dict, axis=0)
concat_df.reset_index(inplace=True)
concat_df.rename(columns={"level_0": "condition", "level_1": "demographic"}, inplace=True)
concat_df['n_case'] = n_patients_df.loc[concat_df['condition'].apply(lambda x: x.lower()), 'case'].values
concat_df['n_control'] = n_patients_df.loc[concat_df['condition'].apply(lambda x: x.lower()), 'control'].values
concat_df['case_rate'] = concat_df['case'] / concat_df['n_case'] * 100
concat_df['control_rate'] = concat_df['control'] / concat_df['n_control'] * 100
concat_df.head()

In [ ]:
with open(result_dir.joinpath("demographics_for_measurement.json")) as file:
    demographics_measurement = json.load(file)

In [ ]:
df_dict = {}
for measurement in demographics_measurement:
    meas_df = pd.DataFrame.from_dict(demographics_measurement[measurement], orient="index")
    meas_df.rename(columns={"False": "control", "True": "case"}, inplace=True)
    df_dict[measurement] = meas_df
df_dict

In [ ]:
concat_df = pd.concat(df_dict, axis=0)
concat_df.reset_index(inplace=True)
concat_df.rename(columns={"level_0": "measurement", "level_1": "demographic"}, inplace=True)

In [ ]:
concat_df['case_mean'] = concat_df['case'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].mean())
concat_df['case_std'] = concat_df['case'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].std())
concat_df['control_std'] = concat_df['control'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].std())
concat_df['control_mean'] = concat_df['control'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].mean())

concat_df['str_case'] = concat_df.apply(lambda x: str(round(x['case_mean'], 2)) + " ± " + str(round(x['case_std'], 2)), axis=1)
concat_df['str_control'] = concat_df.apply(lambda x: str(round(x['control_mean'], 2)) + " ± " + str(round(x['control_std'], 2)), axis=1)

In [ ]:
concat_df[['measurement', 'demographic', 'str_case', 'str_control']].pivot(index='measurement', columns='demographic', values=['str_case', 'str_control']).T

In [ ]:
concat_df = pd.concat(df_dict, axis=0)
concat_df.reset_index(inplace=True)
concat_df.rename(columns={"level_0": "drug", "level_1": "demographic"}, inplace=True)
# extend lists of case and control
concat_df = concat_df.groupby('demographic').apply(lambda x: x.sum())
concat_df.drop(columns=['drug', 'demographic'], inplace=True)
concat_df
concat_df['case_mean'] = concat_df['case'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].mean())
concat_df['case_std'] = concat_df['case'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].std())
concat_df['control_std'] = concat_df['control'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].std())
concat_df['control_mean'] = concat_df['control'].apply(lambda x: np.array(x, dtype=np.float32)[~np.isnan(np.array(x, dtype=np.float32))].mean())   
concat_df['str_case'] = concat_df.apply(lambda x: str(round(x['case_mean'], 2)) + " ± " + str(round(x['case_std'], 2)), axis=1)
concat_df['str_control'] = concat_df.apply(lambda x: str(round(x['control_mean'], 2)) + " ± " + str(round(x['control_std'], 2)), axis=1)
concat_df[['str_case', 'str_control']]